In [ ]:
import pandas as pd
import numpy as np

from data import constants
from data.table_create import ReportTables
from data.table_write_to_excel import WriteMarginTables, WriteReportTables

pd.options.display.max_columns = 100

# ЗП ведомость + Табель

## Таблицы с ведомостями

In [ ]:
report_tables = ReportTables()
tables_for_report = report_tables.get_tables_for_report()
shift_for_report = tables_for_report['shift_for_report']
first_sheet_name = 'salary_record'

for city in shift_for_report['City'].unique():
    salary_table = report_tables.get_salary_table(city)
    file_name = f'{constants.TODAY:%Y-%m-%d}. salary. {city}.xlsx'
    table_of_objects = report_tables.get_table_of_objects(city)
    
    write_report_tables = WriteReportTables(file_name=file_name, first_sheet_name=first_sheet_name)
    write_report_tables.write_weekly_report(city, salary_table)
    
    for month in shift_for_report['month'].unique():
        for obj_id in table_of_objects['ObjectId']:
            if shift_for_report[(shift_for_report['ObjectId'] == obj_id) & (shift_for_report['month'] == month)].shape[0] != 0:
                write_report_tables.write_month_report(tables_for_report=tables_for_report, obj_id=obj_id, month=month)

# Валовая прибыль
### Прошлая неделя

In [ ]:
group_by = ['PartnerName', 'Partner']
column = group_by + ['ResultOfShift_total', 'Price_total', 'Margin', 'Vydanonalichnymi']

gross_margin = shift_result[(shift_result['Type'] == 126) & 
                            (shift_result['Date'] >= constants.START_LAST_WEEK) & 
                            (shift_result['Date'] <= constants.END_LAST_WEEK)]
gross_margin = gross_margin[column].groupby(group_by).agg('sum').reset_index()

gross_margin['City'] = [partners[partners['Id'] == x]['Gorod'].values.item(0)
                        if pd.notna(x) else ''
                        for x in gross_margin['Partner']]
gross_margin['City'] = [base_city[base_city['Id'] == x]['Name'].values.item(0)
                        if pd.notna(x) else ''
                        for x in gross_margin['City']]

drop = ['Partner']
gross_margin.drop(drop, axis=1, inplace=True)

gross_margin.sort_values(by=['Margin'], ascending=False, inplace=True)
gross_margin = gross_margin.rename(columns={'PartnerName': 'Заказчик', 'ResultOfShift_total': 'Себестоимость', 
                                            'Price_total': 'Выручка', 'Margin': 'Валовая прибыль', 'Vydanonalichnymi': 'Выплачено', 
                                            'City': 'Город'})
column = ['Город', 'Заказчик', 'Себестоимость', 'Выручка', 'Валовая прибыль', 'Выплачено']
gross_margin = gross_margin[column]
gross_margin

### Месяц

In [ ]:
group_by = ['PartnerName', 'Partner', 'month']
column = group_by + ['ResultOfShift_total', 'Price_total', 'Margin', 'Vydanonalichnymi']

gross_margin_month = shift_result[(shift_result['Type'] == 126) & 
                            ((shift_result['month'] == constants.MONTH_START) | (shift_result['month'] == constants.MONTH_END))]
gross_margin_month = gross_margin_month[column].groupby(group_by).agg('sum').reset_index()

gross_margin_month['City'] = [partners[partners['Id'] == x]['Gorod'].values.item(0)
                              if pd.notna(x) else ''
                              for x in gross_margin_month['Partner']]
gross_margin_month['City'] = [base_city[base_city['Id'] == x]['Name'].values.item(0)
                              if pd.notna(x) else ''
                              for x in gross_margin_month['City']]

drop = ['Partner']
gross_margin_month.drop(drop, axis=1, inplace=True)

gross_margin_month.sort_values(by=['Margin'], ascending=False, inplace=True)
gross_margin_month = gross_margin_month.rename(columns={'PartnerName': 'Заказчик', 'ResultOfShift_total': 'Себестоимость', 
                                                        'Price_total': 'Выручка', 'Margin': 'Валовая прибыль',
                                                        'Vydanonalichnymi': 'Выплачено', 'City': 'Город'})
column = ['Город', 'Заказчик', 'Себестоимость', 'Выручка', 'Валовая прибыль', 'Выплачено', 'month']
gross_margin_month = gross_margin_month[column]
gross_margin_month

## Записть в Excel

In [ ]:
file_name = f'{constants.TODAY:%Y-%m-%d}. margin.xlsx'
sheet_name = 'gross_margin'

work_book = openpyxl.Workbook()
sheet = work_book.active
sheet.title = sheet_name

work_book.save(file_name)

In [ ]:
work_book = openpyxl.load_workbook(file_name)

for month in gross_margin_month['month'].unique():
    cur_sheet_name = f'{sheet_name}_month_{month}'

    work_book.create_sheet(cur_sheet_name)
    sheet = work_book[cur_sheet_name]

    write_excel.gross_record(gross_margin_month[gross_margin_month['month'] == month], sheet, file_name, work_book, month=month)

sheet = work_book[sheet_name]
write_excel.gross_record(gross_margin, sheet, file_name, work_book)
        
work_book.save(file_name)

In [ ]:
print(f'Скрипт закончил работу. Сформирован файл "{file_name}"')